# Sanity Checks

In [1]:
import os 
import sys
import glob

sys.path.append('/home/kalkiek/projects/reddit-political-affiliation/')

import itertools
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from tqdm.notebook import tqdm
from collections import defaultdict, Counter
from sklearn.metrics import accuracy_score

from src.data.make_dataset import build_dataset
from src.models.word2vec.User2Subreddit import User2Subreddit

from sklearn.metrics import auc, roc_curve

## Distribution of Validation Set

In [2]:
# Load the dataset in
year_month = '2019-04'

network_path = '/shared/0/projects/reddit-political-affiliation/data/bipartite-networks/' + year_month + '_filtered.tsv'
flair_directory = '/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/' + year_month + '.tsv'

dataset, training, validation, pol_validation, vocab = build_dataset(network_path, flair_directory)


Building vocab from file:  20%|█▉        | 5544058/28132858 [00:13<01:14, 304853.36it/s]


Building vocab from file:  40%|███▉      | 11147518/28132858 [00:27<00:35, 477717.97it/s]


Building vocab from file:  60%|█████▉    | 16804471/28132858 [00:40<00:17, 634689.40it/s]


Building vocab from file:  79%|███████▉  | 22183510/28132858 [00:54<00:09, 635267.83it/s]


Building vocab from file:  95%|█████████▌| 26851687/28132858 [01:15<00:38, 33381.70it/s] 


Converting data to PyTorch:   0%|          | 322/5697629 [00:00<29:35, 3208.24it/s]

Length of vocab: 5762539
User count: 5697629
Subreddit count: 64910
User to politic counts: 2589
[('WatchMaga', Counter({'Republican': 2})), ('BasedMedicalDoctor', Counter({'Republican': 2})), ('Damemezaredadremez', Counter({'Republican': 2})), ('joeohegna', Counter({'Republican': 1})), ('Deplorable_scum', Counter({'Republican': 2})), ('TheRabbidHD', Counter({'Republican': 2})), ('Paladin-Arda', Counter({'Democrat': 2})), ('Trumpwillalwayswin', Counter({'Republican': 2})), ('daw-nee-yale', Counter({'Republican': 1})), ('BigcatTV', Counter({'Republican': 2}))]
Saw political affiliations for 2588 users
User to politics training size: {}: 2330
User to politics validation size: {}: 258


Converting data to PyTorch: 100%|██████████| 5697629/5697629 [08:35<00:00, 11053.90it/s]


Train size: 151917434 Validation size: 16879714


In [3]:
reps, dems = 0, 0

for user, label in pol_validation.items():
    if label == 0:
        dems += 1
    else:
        reps += 1

total = reps + dems
print("Percent republicans: {}".format(reps/total))
print("Percent democrats: {}".format(dems/total))

Percent republicans: 0.8992248062015504
Percent democrats: 0.10077519379844961


## Sklearn Dummy Classifiers

In [4]:
user_ids, pol_labels = [], []

for user, pol_label in pol_validation.items():
    try:
        user_ids.append(dataset.user_to_idx[user])
        pol_labels.append(pol_label)
    except KeyError:
        pass

In [5]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(user_ids, pol_labels)
dummy_clf.predict(user_ids)
dummy_clf.score(user_ids, pol_labels)

0.8927038626609443

# Single Month Analysis Not Using Predictions

In [6]:
def read_political_affiliations(files):
    user_to_politic_counts = defaultdict(Counter)

    for fname in tqdm(files):
        with open(fname, 'rt') as f:
            for line in f:
                user, politics, freq = line.split('\t')
                user_to_politic_counts[user][politics] += int(freq)

    print("User to politic counts: " + str(len(user_to_politic_counts)))
    print(list(user_to_politic_counts.items())[:10])

    user_to_politics = {}
    for u, pc in user_to_politic_counts.items():
        if len(pc) > 1:
            continue
        user_to_politics[u] = list(pc.keys())[0]

    print('Saw political affiliations for %d users' % len(user_to_politics))
    return convert_affiliations_to_binary(user_to_politics)


def convert_affiliations_to_binary(user_to_politics):
    for user, politics in user_to_politics.items():
        if politics == "Democrat":
            user_to_politics[user] = 0
        else:
            user_to_politics[user] = 1

    return user_to_politics

flair_directory = "/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/" + year_month + ".tsv"
flair_files = glob.glob(flair_directory)
user_to_politics = read_political_affiliations(flair_files)


User to politic counts: 2589
[('WatchMaga', Counter({'Republican': 2})), ('BasedMedicalDoctor', Counter({'Republican': 2})), ('Damemezaredadremez', Counter({'Republican': 2})), ('joeohegna', Counter({'Republican': 1})), ('Deplorable_scum', Counter({'Republican': 2})), ('TheRabbidHD', Counter({'Republican': 2})), ('Paladin-Arda', Counter({'Democrat': 2})), ('Trumpwillalwayswin', Counter({'Republican': 2})), ('daw-nee-yale', Counter({'Republican': 1})), ('BigcatTV', Counter({'Republican': 2}))]
Saw political affiliations for 2588 users


In [7]:
# Distribution
reps, dems = 0, 0
for k, v in user_to_politics.items():
    if v == 1:
        reps += 1
    else:
        dems += 1
        
total = reps + dems
print("Percent republicans: {}".format(reps/total))
print("Percent democrats: {}".format(dems/total))

Percent republicans: 0.8941267387944358
Percent democrats: 0.10587326120556415


## Polarized Subreddits

In [8]:
user_subreddits = dataset.user_subreddits
subreddit_scores = defaultdict(lambda:0)
subreddit_counts = Counter()
subreddit_users = defaultdict(set)

MIN_POST_THRESHOLD = 20

for user, score in user_to_politics.items():
    subreddits = user_subreddits[user]
    for sub in subreddits:
        if sub[2:4] != 'u_':
            subreddit_scores[sub] += score
            subreddit_counts[sub] += 1
            subreddit_users[sub].add(user)

norm_sub_scores = {}

for sub, score in subreddit_scores.items():
    count = subreddit_counts[sub]
    if count >= 20:
        norm_sub_scores[sub] = score / count
    
sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=True)}
top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

print("Top 25 right leaning subreddits")

for sub, score in top_results.items():
    print(sub, score, subreddit_counts[sub])

Top 25 right leaning subreddits
r/TumblrInAction 1.0 55
r/progun 1.0 59
r/Firearms 1.0 48
r/Catholicism 1.0 32
r/prolife 1.0 29
r/Cringetopia 1.0 42
r/The_MuellerMeltdown 1.0 51
r/drumpfisfinished 1.0 29
r/ShitPoliticsSays 1.0 105
r/TheNewRight 1.0 35
r/tucker_carlson 1.0 21
r/RightwingLGBT 1.0 33
r/benshapiro 1.0 30
r/ar15 1.0 28
r/weekendgunnit 1.0 24
r/gunpolitics 1.0 32
r/walkaway 1.0 29
r/ImGoingToHellForThis 1.0 50
r/Rainbow6 1.0 29
r/gundeals 1.0 45
r/frenworld 1.0 27
r/ZooForTheLowIQRight 1.0 20
r/awfuleverything 1.0 20
r/texas 1.0 21
r/MensRights 1.0 52


In [9]:
sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=False)}
top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

print("Top 25 left leaning subreddits")

for sub, score in top_results.items():
    print(sub, score, subreddit_counts[sub])

Top 25 left leaning subreddits
r/AskALiberal 0.16201117318435754 179
r/centerleftpolitics 0.18181818181818182 22
r/CanadaPolitics 0.358974358974359 39
r/neoliberal 0.45454545454545453 33
r/ENLIGHTENEDCENTRISM 0.5416666666666666 24
r/AskTrumpSupporters 0.5666666666666667 60
r/PoliticalDiscussion 0.6122448979591837 49
r/changemyview 0.6326530612244898 49
r/bestof 0.64 25
r/ProgrammerHumor 0.6451612903225806 31
r/AskConservatives 0.6666666666666666 21
r/AskAnAmerican 0.671875 64
r/canada 0.6756756756756757 37
r/Gamingcirclejerk 0.6956521739130435 23
r/blackmagicfuckery 0.72 25
r/Economics 0.7352941176470589 34
r/TopMindsOfReddit 0.7391304347826086 46
r/europe 0.7391304347826086 23
r/SubredditDrama 0.7419354838709677 31
r/MovieDetails 0.75 28
r/atheism 0.75 44
r/ChapoTrapHouse 0.75 32
r/moderatepolitics 0.7619047619047619 21
r/MHOC 0.7727272727272727 22
r/CFB 0.7777777777777778 27


## Major Subreddit Distributions

In [10]:
import seaborn as sns

major_subreddits = ['r/politics', 'r/pics', 'r/AskReddit', 'r/Conservative', 'r/Liberal']

def plot_sub_scores(subreddit, subreddit_users):
    sns.set_theme(style="darkgrid")
    scores = []
    
    for user in subreddit_users[subreddit]:
        score = user_to_politics[user]
        scores.append(score)
        
    sns.displot(scores).set(title=subreddit)
#     sns.plt.xlim(0, 1)
#     sns.plt.set_title(subreddit)
#     sns.plt.show()

for sub in major_subreddits:
    plot_sub_scores(sub, subreddit_users)


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# SVD On Subreddit Embeddings

In [11]:
from sklearn.decomposition import TruncatedSVD

# Load in the model
PATH = '/shared/0/projects/reddit-political-affiliation/working-dir/word2vec-outputs/' + year_month + '/9.pt'
embedding_dim = 50
model = User2Subreddit(dataset.num_users(), embedding_dim, len(dataset.subreddit_to_idx))
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
print(model.eval())

# Compute SVD on the subreddit embeddings
svd = TruncatedSVD(n_components=10, n_iter=7, random_state=42)
X = model.v_embeddings.weight.detach().numpy()
svd.fit(X)

User2Subreddit(
  (u_embeddings): Embedding(5697629, 50)
  (v_embeddings): Embedding(64910, 50)
  (political_layer): Linear(in_features=50, out_features=1, bias=True)
  (before_pol_dropout): Dropout(p=0.5, inplace=False)
)


TruncatedSVD(algorithm='randomized', n_components=10, n_iter=7, random_state=42,
             tol=0.0)

In [12]:
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)

[0.02673197 0.02344752 0.02207426 0.02170555 0.02122099 0.02109843
 0.0208639  0.02070619 0.02056793 0.02037031]
0.21878704
[100.16848  95.11045  90.69617  89.8216   88.80923  88.55945  88.07457
  87.72496  87.43373  87.01421]


# Multi-month Analysis Not Using Predictions

In [13]:
# Load in multiple months
year_months = ['2019-04', '2019-05', '2019-06']
models = {}
datasets = {}
for year_month in year_months:
    network_path = '/shared/0/projects/reddit-political-affiliation/data/bipartite-networks/' + year_month + '_filtered.tsv'
    flair_directory = '/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/' + year_month + '.tsv'

    dataset, training, validation, pol_validation, vocab = build_dataset(network_path, flair_directory)
    PATH = '/shared/0/projects/reddit-political-affiliation/working-dir/word2vec-outputs/' + year_month + '/9.pt'
    embedding_dim = 50
    model = User2Subreddit(dataset.num_users(), embedding_dim, len(dataset.subreddit_to_idx))
    model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
    models[year_month] = model
    datasets[year_month] = dataset


Building vocab from file:  21%|██▏       | 5989134/28132858 [00:10<00:32, 672108.99it/s]


Building vocab from file:  42%|████▏     | 11712501/28132858 [00:19<00:25, 634276.62it/s]


Building vocab from file:  61%|██████    | 17172417/28132858 [00:48<00:17, 610877.44it/s]


Building vocab from file:  80%|███████▉  | 22377358/28132858 [00:57<00:09, 582339.26it/s]


Building vocab from file:  97%|█████████▋| 27273982/28132858 [01:21<00:01, 508897.71it/s]


...

Building vocab from file:  92%|█████████▏| 28607774/30972308 [01:30<00:04, 562214.11it/s]


Converting data to PyTorch:   0%|          | 286/6071436 [00:00<35:29, 2850.98it/s]

Length of vocab: 6138759
User count: 6071436
Subreddit count: 67323
User to politic counts: 2924
[('CaptChrisPBacon', Counter({'Republican': 2})), ('UndergroundSurface', Counter({'Republican': 2})), ('CharlesEarlBoles', Counter({'Republican': 2})), ('mysteryman64030', Counter({'Republican': 2})), ('derangedwinchester', Counter({'Republican': 2})), ('bro1979', Counter({'Republican': 1})), ('Go_to_the_Astra', Counter({'Republican': 2})), ('guanaco55', Counter({'Republican': 2})), ('DokkaeBeast', Counter({'Republican': 1})), ('PropolisBee', Counter({'Republican': 2}))]
Saw political affiliations for 2919 users
User to politics training size: {}: 2628
User to politics validation size: {}: 291


Converting data to PyTorch: 100%|██████████| 6071436/6071436 [09:59<00:00, 10121.16it/s]


Train size: 167250464 Validation size: 18583384


In [14]:
for year_month in year_months:
    dataset = datasets[year_month]
    model = models[year_month]
    user_subreddits = dataset.user_subreddits
    subreddit_scores = defaultdict(lambda:0)
    subreddit_counts = Counter()
    subreddit_users = defaultdict(set)

    MIN_POST_THRESHOLD = 20

    for user, score in user_to_politics.items():
        subreddits = user_subreddits[user]
        for sub in subreddits:
            if sub[2:4] != 'u_':
                subreddit_scores[sub] += score
                subreddit_counts[sub] += 1
                subreddit_users[sub].add(user)

    norm_sub_scores = {}

    for sub, score in subreddit_scores.items():
        count = subreddit_counts[sub]
        if count >= 20:
            norm_sub_scores[sub] = score / count

    sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=True)}
    top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

    print("Top 25 right leaning subreddits for year: {}".format(year_month))

    for sub, score in top_results.items():
        print(sub, score, subreddit_counts[sub])

    sorted_sub_scores = {k: v for k, v in sorted(norm_sub_scores.items(), key=lambda item: item[1], reverse=False)}
    top_results = dict(itertools.islice(sorted_sub_scores.items(), 25))

    print("Top 25 left leaning subreddits for year: {}".format(year_month))

    for sub, score in top_results.items():
        print(sub, score, subreddit_counts[sub])

Top 25 right leaning subreddits for year: 2019-04
r/TumblrInAction 1.0 55
r/progun 1.0 59
r/Firearms 1.0 48
r/Catholicism 1.0 32
r/prolife 1.0 29
r/Cringetopia 1.0 42
r/The_MuellerMeltdown 1.0 51
r/drumpfisfinished 1.0 29
r/ShitPoliticsSays 1.0 105
r/TheNewRight 1.0 35
r/tucker_carlson 1.0 21
r/RightwingLGBT 1.0 33
r/benshapiro 1.0 30
r/ar15 1.0 28
r/weekendgunnit 1.0 24
r/gunpolitics 1.0 32
r/walkaway 1.0 29
r/ImGoingToHellForThis 1.0 50
r/Rainbow6 1.0 29
r/gundeals 1.0 45
r/frenworld 1.0 27
r/ZooForTheLowIQRight 1.0 20
r/awfuleverything 1.0 20
r/texas 1.0 21
r/MensRights 1.0 52
Top 25 left leaning subreddits for year: 2019-04
r/AskALiberal 0.16201117318435754 179
r/centerleftpolitics 0.18181818181818182 22
r/CanadaPolitics 0.358974358974359 39
r/neoliberal 0.45454545454545453 33
r/ENLIGHTENEDCENTRISM 0.5416666666666666 24
r/AskTrumpSupporters 0.5666666666666667 60
r/PoliticalDiscussion 0.6122448979591837 49
r/changemyview 0.6326530612244898 49
r/bestof 0.64 25
r/ProgrammerHumor 0.645